In [1]:
import os
import numpy as np
# from torch.utils.data import DataLoader, Dataset

from preprocess import resize_input, train_test_split, read_raw
from ear_dataset import EarDataset

import cv2
import torch


In [2]:
n_classes = len(os.listdir("../UERC/")[:500])
n_classes

500

In [3]:
classes = np.arange(1, n_classes+1)

In [4]:
# Randomly choose 30% of the classes
test_mask = np.random.choice([True, False], size=n_classes, p=[0.3, 0.7])

test = classes[test_mask]
train = classes[~test_mask]

len(train), len(test)

(362, 138)

In [5]:
# train_subjects = np.loadtxt("../train_subjects_mask.txt")
# test_subjects = np.loadtxt("../test_subjects_mask.txt")

In [6]:
train_subjects = train
test_subjects = test

In [7]:
np.savetxt("train_500.txt", train)

In [8]:
from PIL import Image
data_path = "../UERC"

In [9]:
ear_data = os.listdir(data_path)

ear_imgs = {}
for person in ear_data:
    if int(person) not in train_subjects:
        continue
    
    imgs = os.listdir("%s/%s" % (data_path, person))
    try:
        ear_imgs[person] = [
            cv2.cvtColor(
                np.asarray(Image.open(f"{data_path}/{person}/{img}")), cv2.COLOR_BGR2RGB
            )
            for img in imgs
        ]
    except Exception as e:
        print(e)

In [10]:
X_train, X_eval, y_train, y_eval = train_test_split(ear_imgs)

In [12]:
len(X_train), len(y_train), len(X_eval), len(y_eval)

(5174, 5174, 2389, 2389)

In [11]:
len(X_train), len(X_eval)

(5174, 2389)

In [13]:
len(set(y_train)), len(set(y_eval))

(343, 343)

### Preprocess data


In [14]:
X_train = resize_input(X_train, tgt_size=64, mode="train")

train_dataset = EarDataset(X_train, y_train)

len(train_dataset.data), len(train_dataset.labels)

/home/sebastijan/anaconda3/envs/torch/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


(5174, 5174)

In [15]:
torch.save(train_dataset, "data/train_dataset_500.pt")


In [16]:
X_eval = resize_input(X_eval, tgt_size=64, mode="test")

eval_dataset = EarDataset(X_eval, y_eval)


In [17]:
len(eval_dataset.data), len(eval_dataset.labels)

(2389, 2389)

In [18]:
torch.save(eval_dataset, "data/eval_dataset_500.pt")


: 